In [1]:
import numpy as np
import time
import cv2
import os

import tensorflow as tf
import tensorflow_hub as hub

# Load Datasets

In [2]:
def load_images_from_folder(folder):
    """
    Loads each image in the given folder and save it in an array.
    
    In case of training data, it saves 3 copies of images:
    original, flipped and rotated copies.
    
    Returns array of images and array of their labels.
    """
    images = []
    labels = []
    classes = 0
    flag = 1
    for file_name in os.listdir(folder):
        if folder == 'Testing': flag = 0 
        sub_dir_path = folder + '/' + file_name
        for filename in os.listdir(sub_dir_path):     
            try:
                img = cv2.imread(os.path.join(sub_dir_path, filename))
                img = cv2.resize(img, (224, 224))
                flipped_img = cv2.flip(img, 0)
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                
            except Exception as e:
                print(' ')
            
            if img is not None:
                images.append(img)
                labels.append(classes)
                if flag:
                    images.append(flipped_img)
                    images.append(rotated_img)
                    labels.append(classes)
                    labels.append(classes)

        classes += 1
    return np.array(images), np.array(labels)

In [3]:
train_images, train_labels = load_images_from_folder("Training")
test_images, test_labels = load_images_from_folder("Testing")
print(len(train_images), len(test_images))

 
 
 
1818 131


In [4]:
print('Training Dataset Shape:{} '.format(train_images.shape)) 
print('No. of Training Dataset Labels:{} '.format(len(train_labels))) 
print('Test Dataset Shape:{} '.format(test_images.shape))
print('No. of Test Dataset Labels:{} '.format(len(test_labels)))

Training Dataset Shape:(1818, 224, 224, 3) 
No. of Training Dataset Labels:1818 
Test Dataset Shape:(131, 224, 224, 3) 
No. of Test Dataset Labels:131 


In [5]:
np.unique(train_images)
train_images = train_images / 255.0
test_images = test_images / 255.0

# Transfer Learning with TF Hub

In [6]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [17]:
model = tf.keras.Sequential([pretrained_model_without_top_layer, tf.keras.layers.Dense(10)])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                12810     
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
_________________________________________________________________


In [8]:
model.compile(optimizer= "adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['acc'])

model.fit(train_images, train_labels, epochs= 5)

Epoch 1/5
57/57 [==============================] - 32s 554ms/step - loss: 0.9700 - acc: 0.7068
Epoch 2/5
57/57 [==============================] - 32s 568ms/step - loss: 0.3032 - acc: 0.9224
Epoch 3/5
57/57 [==============================] - 32s 567ms/step - loss: 0.2119 - acc: 0.9505
Epoch 4/5
57/57 [==============================] - 32s 564ms/step - loss: 0.1617 - acc: 0.9620
Epoch 5/5
57/57 [==============================] - 32s 567ms/step - loss: 0.1326 - acc: 0.9719


In [9]:
model.evaluate(train_images, train_labels)

57/57 [==============================] - 32s 561ms/step - loss: 0.1182 - acc: 0.9719


[0.11817973107099533, 0.971947193145752]

In [10]:
model.evaluate(test_images, test_labels)

5/5 [==============================] - 2s 344ms/step - loss: 0.6500 - acc: 0.8092


[0.6500393748283386, 0.8091602921485901]

In [16]:
# one trial

img = cv2.imread('C:/Users/WIN 10/Documents/Pattern Semester Project/Project/Testing/dna\dna_080.tif')
img = img / 255.0
img = cv2.resize(img, (224, 224))
result = model.predict(img[np.newaxis, ...])
result

array([[-7.609363 ,  3.90926  , -3.836537 , -1.3031648, -3.2525444,
        -3.481797 , -8.16926  , -1.932404 , -1.8995795, -4.3971186]],
      dtype=float32)